## __MLP-Mixer__

In [1]:
import os 
import numpy as np 
import pandas as pd 
import torch 
import torchvision 
import torch.nn as nn 
import torch.optim as optim 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 

from torchvision.transforms import v2 
from tqdm import tqdm 

We will again be implementing our __MLP-Mixer__ model on __CIFAR10__ dataset. This dataset has $60,000$ images of size $32 \times 32$ in 10 classes with $6000$ image per class, so fairly balanced. This dataset, in addition, is already divided into $50,000$ training and $10,000$ test images. 

We will split $5,000$ off of training to create a validation set, leaving training set with $45,000$

In [2]:
#getting test_set first to capture mean and std
from torchvision.datasets import CIFAR10

root = "./data"
test_set = CIFAR10(root=root, train=False, download=True, transform=None)

Files already downloaded and verified


In [ ]:
def compute_mean_std(dataset):
    """"
    To compute Mean and Standard Deviation for each channel
    """

    mean = torch.zeros(3)  
    std = torch.zeros(3)   
    total_pixels = 0

    for image, _ in dataset:
        image = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])(image)  # Converting images to tensors
        image = image.view(3, -1)  # Flattening the image to [3, height*width]
        mean += image.sum(dim=1)   # getting the sum of pixel values for each channel
        std += (image ** 2).sum(dim=1)  #getting the sum of squared pixel values for each channel
        total_pixels += image.size(1)

    # Computing the mean and std across the entire dataset
    mean /= total_pixels
    std = torch.sqrt(std / total_pixels - (mean ** 2))

    return mean.tolist(), std.tolist()

In [6]:
#Getting Mean and Standard Deviation from earlier defined function, 
# and apply transformation using it through Normalize and other augmentation

mean, std = compute_mean_std(test_set)

train_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomHorizontalFlip(),
    v2.RandomResizedCrop((32, 32), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    v2.Normalize(mean=mean, std=std)
])

test_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=mean, std=std)
])

Downloading training set as well, separating validation set out of training set, appling augmentations, and finally creating DATALOADER for each subsets

In [7]:
from torch.utils.data import DataLoader

train_set = CIFAR10(root=root, train=True, transform=train_transform, download=True)

#again same for validation set, this is to different augmentatio application 
validation_set = CIFAR10(root=root, train=True, transform=test_transform, download=True)
test_set = CIFAR10(root=root, train=False, transform=test_transform, download=True)

#separating out distinct train_set and validation set
train_set, _ = torch.utils.data.random_split(train_set, [45000, 5000],generator=torch.Generator().manual_seed(42))
_, validation_set = torch.utils.data.random_split(validation_set, [45000, 5000], generator=torch.Generator().manual_seed(42))


#Next, creating DataLoader(s)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, drop_last=True,num_workers=2, pin_memory=True)
validation_loader = DataLoader(validation_set, batch_size=128, shuffle=True, drop_last=False ,num_workers=2)
test_loader = DataLoader(test_set, batch_size=128, shuffle=True, drop_last=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
